# 将决策问题转化为优化模型

在第一个视频中，我们讨论了数学优化所必需的几个关键概念：
- 参数 
- 决策变量
- 约束条件
- 目标函数

在这第一个建模示例中，我们将看到如何使用这些概念将决策问题表述为优化模型，并使用`gurobipy`编写公式代码。有关Python API中所有命令的更多信息，请查看我们的[文档](https://www.gurobi.com/documentation/10.0/refman/py_python_api_details.html)。

## 决策问题
我们生产小部件。拥有一系列生产设施，这些设施生产装有小部件的箱子。同时还有一系列配送地点，它们将小部件分发销售。每个配送中心都有预测需求，每个生产设施在此期间都有可以制造的小部件数量的最小值和最大值。我们需要确保每个配送设施从生产中接收到足够的小部件以满足需求，并且我们希望以最低成本做到这一点。最小生产量是生产设施最大值的75%。

## 集合和定义模型
我们的集合是：
- $P = \{\texttt{'Baltimore', 'Cleveland', 'Little Rock', 'Birmingham', 'Charleston'}\} \quad\quad\quad\quad\quad\quad\quad\space\space \texttt{production}$
- $D = \{\texttt{'Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St. Louis'}\} \quad\quad\quad \texttt{distribution}$

为了索引每个集合，我们将使用每个集合的小写字母。用于集合和索引的字母由您决定。通常，大写字母用于集合，相应的小写字母将用作索引。使用单个字母主要是为了简洁。

In [1]:
# %pip install gurobipy

# 首先，导入包
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# 分别设置P和D
# 当我们对集合进行编码时，我们可以在名称中更具描述性
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis']

# 为决策问题定义一个gurobipy模型
m = gp.Model('widgets')

Set parameter LicenseID to value 2601452


## 参数

数学优化问题的参数是在模型中被视为常量的值，并与决策变量相关联。对于这个决策问题，这些值是每个生产设施的限制，每个配送中心的需求，以及生产和配送位置之间的成对成本。

- $m_p$ 是位置 $p$ 的最大生产量，$\forall p \in P \quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\space\space \texttt{max}\_\texttt{prod[p]}$
- $n_d$ 是配送中心 $d$ 的客户数量，$\forall d \in D \quad\quad\quad\quad\quad\quad\quad\quad \texttt{n}\_\texttt{demand[d]}$
- $c_{p,d}$ 是将小部件从位置 $p$ 运送到位置 $d$ 的成本，$\forall p \in P, d \in D \quad\quad\quad \texttt{cost[p,d]}$

In [2]:
# 使用.squeeze（“columns”）使成本成为一个系列
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1]).squeeze("columns")
# transp_cost = pd.read_csv('cost.csv', index_col=[0,1]).squeeze("columns")
# Pivot可以更容易地查看成本
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St. Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [3]:
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 
max_prod.to_frame()
#n_demand.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little Rock,140
Birmingham,80
Charleston,180


我们还有一个要求，即每个生产设施需要生产最大产能的75%。在公式中，我们将这个值表示为 $a$，"frac"表示所需最大产量的比例。初始设定 $a = 0.75$。

In [4]:
frac = 0.75

## 决策变量
这是优化求解器确定的内容，也就是您可以控制的操作。提醒一下，它们主要有三种类型：
- `连续型`：产品价格
- `整数型`：用于活动的食品卡车数量
- `二元型`：在投资组合中是否包含某种股票的决策

决策变量（和参数）使用我们为问题定义的集合元素进行索引。在这个例子中，让我们从生产我们小部件的城市集合开始，在公式中我们称之为集合 $P$，但在代码中可以定义为'production'。同样，分发小部件的城市集合 $D$ 和'distribution'。这里的决策是确定从每个生产设施发送到每个配送地点的盒子数量。

令 $x_{p,d}$ 表示在设施 $i$ 生产并运送到位置 $j$ 的小部件数量。

### 在gurobipy中添加变量
`gurobipy` 主要通过两个（相似的）命令让您添加决策变量：
- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) 添加单个变量
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) 通过集合/索引添加一组变量

使用 `addVars` 时，您必须提供要添加的变量的索引，对我们而言，就是生产和配送地点。还有其他参数可以使用，我们稍后会介绍其中的几个。

### 我们的决策变量
正如编写代码时经常发生的情况，有几种方法可以达到同一个目的。下面我们可以看到创建决策变量的三种不同方式。

In [5]:
# 循环遍历每个p和d组合以创建一个决策变量
m = gp.Model('widgets')
x = {}
for p in production:
    for d in distribution:
        x[p,d] = m.addVar(name = p+"_to_"+d)
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var Baltimore_to_Columbia>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var Baltimore_to_Indianapolis>,
 ('Baltimore', 'Lexington'): <gurobi.Var Baltimore_to_Lexington>,
 ('Baltimore', 'Nashville'): <gurobi.Var Baltimore_to_Nashville>,
 ('Baltimore', 'Richmond'): <gurobi.Var Baltimore_to_Richmond>,
 ('Baltimore', 'St. Louis'): <gurobi.Var Baltimore_to_St. Louis>,
 ('Cleveland', 'Columbia'): <gurobi.Var Cleveland_to_Columbia>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var Cleveland_to_Indianapolis>,
 ('Cleveland', 'Lexington'): <gurobi.Var Cleveland_to_Lexington>,
 ('Cleveland', 'Nashville'): <gurobi.Var Cleveland_to_Nashville>,
 ('Cleveland', 'Richmond'): <gurobi.Var Cleveland_to_Richmond>,
 ('Cleveland', 'St. Louis'): <gurobi.Var Cleveland_to_St. Louis>,
 ('Little Rock', 'Columbia'): <gurobi.Var Little Rock_to_Columbia>,
 ('Little Rock', 'Indianapolis'): <gurobi.Var Little Rock_to_Indianapolis>,
 ('Little Rock', 'Lexington'): <gurobi.Var Little Rock_to_Le

In [6]:
# 为索引提供每个集合
m = gp.Model('widgets')
x = m.addVars(production, distribution, name = 'prod_ship')
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

In [ ]:
# 运输成本指标有生产地点和配送地点的组合
m = gp.Model('widgets')
x = m.addVars(transp_cost.index, name = 'prod_ship')
m.update()
x

{('Baltimore', 'Columbia'): <gurobi.Var prod_ship[Baltimore,Columbia]>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var prod_ship[Baltimore,Indianapolis]>,
 ('Baltimore', 'Lexington'): <gurobi.Var prod_ship[Baltimore,Lexington]>,
 ('Baltimore', 'Nashville'): <gurobi.Var prod_ship[Baltimore,Nashville]>,
 ('Baltimore', 'Richmond'): <gurobi.Var prod_ship[Baltimore,Richmond]>,
 ('Baltimore', 'St. Louis'): <gurobi.Var prod_ship[Baltimore,St. Louis]>,
 ('Cleveland', 'Columbia'): <gurobi.Var prod_ship[Cleveland,Columbia]>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var prod_ship[Cleveland,Indianapolis]>,
 ('Cleveland', 'Lexington'): <gurobi.Var prod_ship[Cleveland,Lexington]>,
 ('Cleveland', 'Nashville'): <gurobi.Var prod_ship[Cleveland,Nashville]>,
 ('Cleveland', 'Richmond'): <gurobi.Var prod_ship[Cleveland,Richmond]>,
 ('Cleveland', 'St. Louis'): <gurobi.Var prod_ship[Cleveland,St. Louis]>,
 ('Little Rock', 'Columbia'): <gurobi.Var prod_ship[Little Rock,Columbia]>,
 ('Little Rock',
  'Indianapolis

命令 `m.update()` 更新模型以包含已经进行的任何更改，如添加变量。它不需要在每个单元格中运行，但如果您在单元格的输出中看到 *等待模型更新*，那么这应该可以防止这种情况发生。

## 约束条件
我们在本示例开始时概述了生产和需求约束；现在我们将其进行公式化并编码。注意，向模型中添加约束条件（和/或决策变量）的顺序并不重要。

### 在gurobipy中添加约束条件
向模型添加约束条件与添加变量类似：
- [addConstr()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstr.html) 添加单个约束
- [addConstrs()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstrs.htmll) 使用Python `生成器` 表达式添加一组约束
 
### 我们的约束条件
首先，我们将为每个配送地点制定需求约束，并将其添加到模型中。

\begin{align*} 
\sum_{p}x_{p,d} \ge n_d, \quad \forall d \in D \quad\quad \texttt{meet}\_\texttt{demand[d]}\\ 
\end{align*}

这将是我们第一次使用 [gp.quicksum()](https://www.gurobi.com/documentation/10.0/refman/py_quicksum.html)。在gurobipy中有其他方法可以对表达式求和，虽然这种方法在编码上不是最简洁的，但很容易将其与公式中的求和进行比较，以了解其工作原理。

In [8]:
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')
#meet_demand = m.addConstrs(x.sum('*', j) >= demand[j] for j in distribution)
m.update()
meet_demand

{'Columbia': <gurobi.Constr meet_demand[Columbia]>,
 'Indianapolis': <gurobi.Constr meet_demand[Indianapolis]>,
 'Lexington': <gurobi.Constr meet_demand[Lexington]>,
 'Nashville': <gurobi.Constr meet_demand[Nashville]>,
 'Richmond': <gurobi.Constr meet_demand[Richmond]>,
 'St. Louis': <gurobi.Constr meet_demand[St. Louis]>}

接下来，我们有每个生产设施可以制造的最大小部件数量。我们还规定每个设施必须至少生产其最大产能的75%。

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P \quad\quad &\texttt{can}\_\texttt{produce[p]}\\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P \quad\quad &\texttt{must}\_\texttt{produce[p]}\\ 
\end{align*}
$$


In [9]:
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')# (x.sum(i, '*')
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')
m.update()
can_produce

{'Baltimore': <gurobi.Constr can_produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can_produce[Cleveland]>,
 'Little Rock': <gurobi.Constr can_produce[Little Rock]>,
 'Birmingham': <gurobi.Constr can_produce[Birmingham]>,
 'Charleston': <gurobi.Constr can_produce[Charleston]>}

## 目标函数
我们被告知要**降低**运输成本，我们将用这个来确定我们的目标函数，即最小化从生产地点到配送地点运送小部件的总成本。

### 在gurobipy中设置目标
这是通过 [setObjective()](https://www.gurobi.com/documentation/10.0/refman/py_model_setobjective.html) 完成的。第二个参数（本例中为 `GRB.MINIMIZE`）称为模型的*方向*。对于最大化问题，我们将使用 `GRB.MAXIMIZE`。

### 我们的目标函数
\begin{align*} 
{\rm minimize} \space \sum_{p,d}c_{p,d}x_{p,d}, \quad \forall p \in P, d \in D\\ 
\end{align*}

In [10]:
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE) 

## 查找、提取和分析解决方案
在运行优化之前，编写一个 `lp` 文件是个好主意。这是一个文本文件，它打印出变量、约束条件和对象，就像我们在*公式*中看到的那样，只是没有求和符号，而是使用我们指定的名称。

In [11]:
m.write('widget_shipment.lp')

### 运行优化

In [11]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.704890000e+03


### 提取解决方案
有很多方法可以从gurobipy中获取决策变量的值。

In [12]:
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
#sol 
sol[sol.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

这里还有几种获取解决方案的方法。

In [16]:
# 您可以获取所有决策变量的名称和值：
all_vars = {v.varName: v.x for v in m.getVars()} 
all_vars

{'prod_ship[Baltimore,Columbia]': 0.0,
 'prod_ship[Baltimore,Indianapolis]': 0.0,
 'prod_ship[Baltimore,Lexington]': 0.0,
 'prod_ship[Baltimore,Nashville]': 19.0,
 'prod_ship[Baltimore,Richmond]': 116.0,
 'prod_ship[Baltimore,St. Louis]': 0.0,
 'prod_ship[Cleveland,Columbia]': 89.0,
 'prod_ship[Cleveland,Indianapolis]': 95.0,
 'prod_ship[Cleveland,Lexington]': 0.0,
 'prod_ship[Cleveland,Nashville]': 2.0,
 'prod_ship[Cleveland,Richmond]': 0.0,
 'prod_ship[Cleveland,St. Louis]': 0.0,
 'prod_ship[Little Rock,Columbia]': 0.0,
 'prod_ship[Little Rock,Indianapolis]': 0.0,
 'prod_ship[Little Rock,Lexington]': 0.0,
 'prod_ship[Little Rock,Nashville]': 0.0,
 'prod_ship[Little Rock,Richmond]': 0.0,
 'prod_ship[Little Rock,St. Louis]': 140.0,
 'prod_ship[Birmingham,Columbia]': 0.0,
 'prod_ship[Birmingham,Indianapolis]': 0.0,
 'prod_ship[Birmingham,Lexington]': 0.0,
 'prod_ship[Birmingham,Nashville]': 80.0,
 'prod_ship[Birmingham,Richmond]': 0.0,
 'prod_ship[Birmingham,St. Louis]': 0.0,
 'prod_shi

或者您可以只遍历特定变量并仅返回您感兴趣的值。记住，在python中x是一个字典。因此，像遍历任何字典一样遍历它

In [17]:
xvals = {k: v.x for k,v in x.items() if v.x > 0} 
xvals 

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little Rock', 'St. Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St. Louis'): 41.0}

In [18]:
x_values.sum(), n_demand.sum()

(703.0, 703)

### 解决方案分析
虽然确定小部件的最佳运输方案是我们的目标，但我们可能希望更深入地研究解决方案。例如，我们可以按设施汇总总生产量，以查看哪些地点（如果有）未达到其最大小部件生产能力，以及哪些（如果有）生产设施处于其生产下限。

In [19]:
# 按生产设施汇总装运量
ship_out = sol.groupby('production')['shipment'].sum()
pd.DataFrame({'Remaining':max_prod - ship_out, 'Utilization':ship_out/max_prod})

,Remaining,Utilization
Baltimore,45.0,0.75
Birmingham,0.0,1.00
Charleston,18.0,0.90
Cleveland,14.0,0.93
Little Rock,0.0,1.00


在数学优化中，当不等式约束的左侧和右侧相等时，我们说该约束是`约束性的`（binding）。当这种情况*没有发生*时，该约束中就存在`松弛`（slack）或`剩余`（surplus）。我们可以通过调用约束的`Slack`属性来获取此值。

In [20]:
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production], 
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]}, 
             index = production)

,Remaining,Utilization
Baltimore,45.0,0.75
Cleveland,14.0,0.93
Little Rock,0.0,1.00
Birmingham,0.0,1.00
Charleston,18.0,0.90


# 使用二元变量
正如我们在第一节课和本笔记本开头所描述的那样，二元变量用于在数学优化中选择备选方案。它们可以被解释为是/否决策或开/关开关。

在原问题中，伯明翰的产能远低于其他设施。假设我们有选择将该设施的最大产能增加25个或50个小部件的选项，但选择其中一个选项分别需要支付50美元和75美元的成本，并且我们最多只能选择一个。我们将为每个选项使用一个名为$xprod$的二元决策变量。

令$xprod_0 = 1$表示我们选择第一个选项并将产能增加25个小部件，否则为$0$。
令$xprod_1 = 1$表示我们选择第二个选项并将产能增加50个小部件，否则为$0$。

虽然使用单个小写字母作为决策变量相当常见，但这并非必要，您会经常看到像上面那样定义的变量（更具描述性）。我们将制定一个新模型，其中包含与之前相同的决策变量和需求约束。

In [21]:
# 我们用m2作为第二个模型
# 这些部件除新型号名称外与上述相同
m2 = gp.Model('widgets2')
x = m2.addVars(production, distribution, obj = transp_cost, name = 'prod_ship')
meet_demand = m2.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')

在上面的单元格中，我们确实使用了`addVars()`函数的新参数：`obj`。这将设置添加到目标函数中的决策变量的系数，等同于我们之前通过将每个生产地点和配送地点之间的运输成本附加到适当的决策变量所做的工作。

接下来，我们将为除伯明翰以外的每个生产设施添加与之前相同的生产限制约束。除了约束条件适用的集合外，公式基本相同。
$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p, &\forall p \in P -\{\texttt{Birmingham}\} \\ 
\sum_{d}x_{p,d} &\ge a*m_p,&\forall p \in P -\{\texttt{Birmingham}\} \\ 
\end{align*}
$$

在gurobipy中，这是通过在`生成器`表达式中添加条件来完成的。

In [22]:
can_produce = m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production if p != 'Birmingham'), name = 'can_produce')
must_produce =  m2.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production if p != 'Birmingham'), name = 'must_produce')

现在，添加新的二元变量。

In [23]:
xprod = m2.addVars(range(2), vtype = GRB.BINARY, obj = [50, 75], name = 'expand_Birmingham_prod')

让我们分解上面单元格中的每个参数 -- 那里有一些新内容。
1. `range(n)`用于添加$n$个决策变量。在本例中，我们添加了2个变量。
2. 我们需要使用`vtype`将其声明为二元变量。
3. 我们再次使用`obj`功能立即设置这些变量的目标函数系数。

目标和新的二元变量在公式中看起来像这样：

\begin{align*} 
{\rm minimize} \space &\sum_{p,d}c_{p,d}x_{p,d} + 50*xprod_0 + 75*xprod_1, \quad &\forall p \in P, d \in D\\ 
&xprod_i \in \{0,1\}, &{\rm for} \space i \in \{0,1\}
\end{align*}

接下来，我们有特定于伯明翰设施的生产约束。

$$
\begin{align*} 
\sum_{d}x_{p,d} &\le m_p + 25*xprod_0 + 50*xprod_1, & p = \texttt{Birmingham} \\ 
\sum_{d}x_{p,d} &\ge a*(m_p+ 25*xprod_0 + 50*xprod_1),& p = \texttt{Birmingham} \\ 
\end{align*}
$$

In [24]:
Birmingham_max = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) <= max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1], name = 'Birmingham_add_max')
Birmingham_min = m2.addConstr(gp.quicksum(x['Birmingham',d] for d in distribution) >= frac*(max_prod['Birmingham'] + 25*xprod[0] + 50*xprod[1]), name = 'Birmingham_add_min')

上面提到，我们最多可以选择一个扩展选项，这意味着我们不能允许$xprod_0$和$xprod_1$同时等于1。为了对此建模，我们添加了一个约束，将这两个二元变量的总和限制为最多一个。

$$
\begin{align*}
\sum_{i}xprod_i \le 1
\end{align*}
$$
在gurobipy中相应的约束：

In [25]:
Birmingham_lim = m2.addConstr(gp.quicksum(xprod[i] for i in range(2)) <= 1, name = 'expansion_choice')

现在我们可以运行这个优化模型，看看这种潜在的扩展是否有助于我们降低总体成本。

In [26]:
m2.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 17 rows, 32 columns and 96 nonzeros
Model fingerprint: 0x80ef8db6
Variable types: 30 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [2e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve time: 0.00s
Presolved: 17 rows, 32 columns, 96 nonzeros
Variable types: 30 continuous, 2 integer (2 binary)
Found heuristic solution: objective 1704.8900000

Root relaxation: objective 1.694760e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1694.76000    0    1 1704.89000 1694.76000 

In [27]:
obj1 = m.getObjective()
obj2 = m2.getObjective()
print(f"原始模型的总成本为 {round(obj1.getValue(),2)}")
print(f"新公式的总成本为 {round(obj2.getValue(),2)}")

原始模型的总成本为 1704.89
新公式的总成本为 1700.4


目标函数值的变化告诉我们什么？

让我们看看我们的二元变量的值。

In [28]:
pd.Series(m2.getAttr('X', xprod))

0    1.0
1    0.0
dtype: float64

模型选择了第一个扩展选项，因为$xprod_0 = 1$，这意味着在伯明翰增加25个小部件的产能。我们可以看到其余的解决方案，其中将包括伯明翰产能的增加。

In [29]:
x2_values = pd.Series(m2.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x2_values], axis=1)
sol2[sol2.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Richmond      1.96     135.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53     101.0
            St. Louis     4.01       4.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      37.0

In [30]:
m.dispose()
m2.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


### 作业！（不是真正的作业，但值得研究）
分析两个模型之间的最优解如何变化。你会注意到一些奇怪的现象。
- 奇怪的是什么？
- 你认为为什么会发生这种情况？
- 从公式角度和业务角度，你将如何解决这个问题？